# Two-dimensional RTE dataset notebook

This dataset consists of solutions of the following transport equation:

$$
    \bm{v}\cdot\nabla f(\bm{x},\bm{v}) + \sigma(\bm{x})f(\bm{x},\bm{v}) = \sigma_s(\bm{x})\int_{\mathbb{S}^{d-1}}k(\bm{v},\bm{v}')f(\bm{x},\bm{v}')\, d\bm{v}'，
$$

which is defined on a rectangle spatial domain $\Omega = [0, 1]\times[0, 1]\subset \mathbb{R}^d$ and velocity/momentum space $\mathbb{S}^{d-1}$.

The integration area is $\Omega = [0,1]\times[0,1]$, The boundary conditions of the problem are

$$
    \left\{\begin{array}{l}
        f_L(y,ct,st) = \frac{1}{(2\pi\sigma)^{\frac{3}{2}}}e^{-\frac{(y-y_l')^2}{2\sigma}}e^{-\frac{(ct-ct_l')^2+(st-st_l')}{2\sigma}}, \\
        f_R(y,ct,st) =  \frac{1}{(2\pi\sigma)^{\frac{3}{2}}}e^{-\frac{(y-y_r')^2}{2\sigma}}e^{-\frac{(ct-ct_r')^2+(st-st_r')}{2\sigma}}, \\
        f_B(x,ct,st) = \frac{1}{(2\pi\sigma)^{\frac{3}{2}}}e^{-\frac{(x-x_b')^2}{2\sigma}}e^{-\frac{(ct-ct_b')^2+(st-st_b')}{2\sigma}}, \\
        f_T(x,ct,st) = \frac{1}{(2\pi\sigma)^{\frac{3}{2}}}e^{-\frac{(x-x_t')^2}{2\sigma}}e^{-\frac{(ct-ct_t')^2+(st-st_t')}{2\sigma}}
    \end{array}\right.
$$

where $y_l',y_r',x_b',x_t'$ is randomly sampled from the 0th grid point in the y,x direction to the 40th grid point and obeys uniform distribution,$\sigma$ obey the uniform distribution between $[0.001,0.021]$.
The parameters are set as follows:

$$
    \left\{\begin{array}{l}
        \sigma_{T}=5+n_T, \quad \sigma_{a}=2+n_a, \quad q=0, \quad \varepsilon=1, \quad (x,y) \in [0.4,0.6]\times [0.4,0.6] \\
        \sigma_{T}=10, \quad \sigma_{a}=5, \quad q=0, \quad \varepsilon=1 . \quad \text{otherwise}
    \end{array}\right.
$$

where $n_T,n_a$ are random noise, obey the uniform distribution between $[-2,2]$

In the numerical solution program, 500 pieces of data are generated, and $N_x=N_y=40,N_v=24$.


In [1]:
%pip install matplotlib plotly nbformat

     |████████████████████████████████| 11.9 MB 468 kB/s            
     |████████████████████████████████| 26.5 MB 216 kB/s             
     |████████████████████████████████| 178 kB 45.8 MB/s            
     |████████████████████████████████| 40 kB 834 kB/s             
     |████████████████████████████████| 890 kB 43.6 MB/s            
     |████████████████████████████████| 97 kB 1.0 MB/s             
     |████████████████████████████████| 1.6 MB 38.6 MB/s            
     |████████████████████████████████| 4.3 MB 41.5 MB/s            
     |████████████████████████████████| 72 kB 216 kB/s             
     |████████████████████████████████| 115 kB 45.9 MB/s            
Note: you may need to restart the kernel to use updated packages.


## key 值说明

- I:
  - x 方向上的离散点数。
  - shape: (1, 1)
  - 值：40
- J:
  - y 方向上的离散点数。
  - shape: (1, 1)
  - 值：40
- M

  - 速度离散
  - shape: (1, 1)

- list_Phi
  - 密度$\rho$
  - shape: (40, 40, 500)
- list_Psi

  - 方程解$f(x,v)$在网格点上的值。
  - shape: (24, 40, 40, 500)

- list_psiB
  - 方程在下边界的对应值，$f(x,y_B,v)$
  - shape:(12, 40, 500)
- list_psiL
  - 方程在左边界的对应值，$f(x_L,y,v)$
  - shape:(12, 40, 500)
- list_psiR
  - 方程在右边界的对应值，$f(x_R,y,v)$
  - shape:(12, 40, 500)
- list_psiT
  - 方程在上边界的对应值，$f(x,y_T,v)$
  - shape:(12, 40, 500)
- list_q
  - $q(x,y)$源项的值。
  - shape:(40, 40, 500)
- list_sigma_T
  - $\sigma_T(x,y)$源项的值。
  - shape:(40, 40, 500)
- list_sigma_a
  - $\sigma_a(x,y)$源项的值。
  - shape:(40, 40, 500)
- list_varepsilon
  - $\varepsilon(x,y)$源项的值。
  - shape:(40, 40, 500)
- omega (1, 24)
  - v 的高斯勒让德多项式零点对应的权重。
- theta (1, 24)
  - v 的离散点。


In [2]:
import numpy as np
import tensorflow as tf
import scipy.io as sio


In [3]:
def load_mat(filename):
    data = sio.loadmat(filename)
    for key in data:
        if type(data[key]) == np.ndarray and len(data[key].shape) != 1:
            data[key] = np.moveaxis(data[key], -1, 0).astype(np.float32)
    return data


In [4]:
def save_as_npy(filename_list, filename="../data/2d_RTE.npy"):
    data_dr = load_mat(filename_list)
    data_dr["list_Psi"] = np.moveaxis(data_dr["list_Psi"], 1, -1)
    np.save(filename, data_dr)


In [5]:
data_path = "matlab/e1_L_delta_full_1.mat"
data1 = load_mat(data_path)
data1["list_Psi"] = np.moveaxis(data1["list_Psi"], 1, -1)


In [15]:
import plotly.graph_objects as go

idx = np.random.randint(500)

n = 40
h = 1 / n
z = data1["list_Phi"][idx]
x = y = np.linspace(0 + 3 * h / 2, 1 - 3 * h / 2, 40)
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(
    autosize=False,
    margin=dict(l=65, r=50, b=65, t=90),
)
fig.show()
